## **Diffusion Equation**

$$
\frac{d\Sigma}{dt} = \frac{3}{r}\frac{\partial}{\partial r} \left[r^{1/2} \frac{\partial}{\partial r}(\nu \Sigma r^{1/2})\right]
$$

This is evolution equation for the surface density of a geometrically thin disk

Putting this in the form of a diffusion equation:

$$
\frac{\partial f}{\partial t} = D \frac{\partial^2 f}{\partial X^2}
$$
where $X \equiv 2r^{1/2}$, $f \equiv \frac{3}{2}\Sigma X$ and $D = \frac{12\nu}{X^2}$


We need numerical values for $\nu$
$$
\nu = \alpha c_s H \\
H = \frac{c_s}{\Omega} \\
\Omega = \sqrt{\frac{G*M_s}{r^3}} \\
c_s = \sqrt{\frac{k_B * T}{\mu * mH}} \\
mH = \frac{1}{N_A} \\
\mu = 2.3 \\
$$

The Temperature profile is given by:
$$
2 \sigma_b T_{disk}^4 = \frac{9}{4}\Sigma \nu \Omega^2\\
T^3 = \frac{9}{8\sigma_b}\Sigma \frac{k_b \alpha}{\mu mH}\sqrt{\frac{GM}{r^3}}
$$

The Pressure is given by:

$$
P = \frac{c_s^2\Sigma}{H} = c_s\Sigma\Omega
$$


$$St_{max} = \frac{Q*}{\alpha c_s^2}$$


## **Pebble Accretion**

Mass Accretion is given by:
$\dot{M}_{3D} = \pi R_{acc}\rho_{p}\delta v$

Accretion Criterion:
$\tau_f = \cfrac{\xi_B \Delta v + \xi_H \Omega \hat{R}_{acc}}{GM/\hat{R}^2_{acc}}$

$\hat{R}^2_{acc} = \left(\frac{4\tau_f}{t_B}\right)^{1/2} R_B$

$R_B = \frac{GM}{\Delta v^2}$

$t_B = \frac{R_B}{\Delta v}$

$\tau_f = \frac{St}{\Omega}$

$\xi_B = \xi_H = 0.25$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba
import pandas as pd
import os
import sys
from dust_diffuse import *
from thermal import *
import matplotlib.cm as cm
import matplotlib.patches as ptch
from matplotlib import rc
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import pylab
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings
import csv

In [2]:
#Natural Constants
# T = 600 # Temperature
mu = 2.2 # mean molar mass in disk
avog = 6.02214 * 10**23 # avogadros number
mH = 1.673534 * 10**-27 # atomic hydrogen mass
kB = 1.380649 * 10**-23 #boltzmann constant
G = 6.6738 * 10**-11 #gravitational constant
Ms = 1.9886 * 10**30 #solar mass
Me = 5.972 * 10 **24 #earth mass
AU = 1.4959787 * 10**11 #astronomical units
yrs2sec = 3.1536 * 10**7 #convert years to seconds
sb = 5.6704*10**-8

In [3]:
#Disk Parameters
alpha1 = 1e-2
alpha2 = 1e-3
alpha3 = 1e-4
rin = 0.05 * AU #inner radius of the disk
rout = 30 * AU #outer radius of the disk
sigma_in  = 2 * 10**5 #boundary condition at t=0
sigma_max = sigma_in*2 #boundary condition at t=final_time
sigma_min = 1 * 10**2
distance = rout - rin #distance from inner radius to outer radius

In [85]:
#Temporal discretization
max_years = 4
dyr = .1
dt = dyr * yrs2sec #timestep
final_time = max_years*yrs2sec + 1 #total diffusion time in seconds
t_save_interval = 1
t = np.arange(3*yrs2sec, final_time, dt)

'2'

In [6]:
#Other Parameters

# St = .01 #Stokes Number
Q = 1
Tcor = 1800 #10% corundum
Tcalc = 1700 #10% calcium
Tfor = 1400 #80% forsterite

cor_ratio = .1
calc_ratio = .1
for_ratio = .8

phase_change = 1600

In [7]:
#Initialize grid spaces

dist = np.empty(n)
Omega = np.empty(n)
v_phi_dust = np.empty(n)
X = np.empty(n)

In [8]:
#Initialize Staggered Grid

dist_stag = np.empty(n-1)
Omega_stag = np.empty(n-1)

In [9]:
#Boundary Conditions

f_in = 0
f_out = 0

In [10]:
@numba.njit()
def make_alpha_grid(cont):
    if(cont):
        alphas = np.linspace(alpha1, alpha3, n)
    else:
        alphas = np.empty(n)
        for i in range(n):
            if i < (n//3):
                alphas[i] = alpha1
            elif i < (2*n//3):
                alphas[i] = alpha2
            else:
                alphas[i] = alpha3
    return alphas

In [11]:
def calc_dist_omega_stag():
    for i in range(1, n):
        dist_stag[i-1] = 0.5 * (dist[i] + dist[i-1])
        Omega_stag[i-1] = np.sqrt(G * Ms / dist_stag[i-1] ** 3)

In [12]:
def update_stag_grid(alphas, cs2):
    cs2_stag = np.array([0.5 * (cs2[i] + cs2[i-1]) for i in range(1, n)])
    alphas_stag = np.array([0.5 * (alphas[i] + alphas[i-1]) for i in range(1, n)])
    return (alphas_stag, cs2_stag)

In [13]:
@numba.njit
def update_nu(cs2, alphas):
    return alphas*cs2/Omega

In [14]:
@numba.njit
def update_St(cs2, cs2_stag, alphas, alpha_stag):
    dust = Q/(cs2_stag*alpha_stag)
    pebble = Q*10/(cs2*alphas)
    return (dust, pebble)

In [15]:
@numba.njit
def update_D(nu):
    return 12 * nu / (X ** 2)

In [16]:
def calc_dust_vel(St, cs2_stag, v_gas, press):
    press_stag = [0.5*(press[i]+press[i-1]) for i in range(1,n)]
    dPdr_stag = [(press[i] - press[i-1])/dist_stag[i-1] for i in range(1,n)]
    rho_stag = press_stag/cs2_stag
    v_dust = (St/(1+(St)**2)) * (1 / (rho_stag * Omega_stag))*dPdr_stag
    v_dust = np.append(v_dust, 0)
#     print('cs2_stag',cs2_stag)
#     v_dust = v_dust + v_gas
    return v_dust

In [17]:
def calc_v_phi_gas(press, cs2):
    dPdr = np.zeros(n)
    for i in range(n):
        if (i == 0):
            dPdr[i] = (press[i+1] - press[i])/(dist[i+1]-dist[i])
        elif(i == n-1):
            dPdr[i] = (press[i] - press[i-1])/(dist[i]-dist[i-1])
        else:
            dPdr[i] = (press[i+1] - press[i-1])/(2*(dist[i+1]-dist[i-1]))
    rho = press/cs2
    temp = (G*Ms/(dist)) + (dist/rho)*dPdr
    for idx, val in enumerate(temp):
        if (val < 0):
            temp[idx] = v_phi_dust[idx]
    v_phi_gas = np.sqrt(temp)
    return v_phi_gas

In [18]:
def calc_init_params(*folder):
    """
    Calculate the initial parameters values for time t = 0.
    """
    for i in range(n):
        dist[i] = (rin + (rout-rin)*i/(n-1))
        X[i] = 2 * np.sqrt(dist[i])
        Omega[i] = np.sqrt(G * Ms / (dist[i] ** 3))
        v_phi_dust[i] = np.sqrt(Omega[i]**2 * dist[i])
    if(folder):
        df = pd.read_csv(folder[0] + '/disk_' + str(int(t[0]/yrs2sec) - t_save_interval) + '.txt', delimiter=',', header=None)
        sigma_gas = df[1].to_numpy().astype(np.float)
        cor_dust = df[2].to_numpy().astype(np.float)
        calc_dust = df[3].to_numpy().astype(np.float)
        for_dust = df[4].to_numpy().astype(np.float)
        cor_evap = df[5].to_numpy().astype(np.float)
        calc_evap = df[6].to_numpy().astype(np.float)
        for_evap = df[7].to_numpy().astype(np.float)
        cor_peb = df[8].to_numpy().astype(np.float)
        calc_peb = df[9].to_numpy().astype(np.float)
        for_peb = df[10].to_numpy().astype(np.float)
        T = df[12].to_numpy().astype(np.float)
        P = df[13].to_numpy().astype(np.float)

        calc_dist_omega_stag()

        sigma_dust = np.vstack([cor_dust,calc_dust,for_dust])
        sigma_evap = np.vstack([cor_evap,calc_evap,for_evap])
        sigma_peb = np.vstack([cor_peb,calc_peb,for_peb])

        alphas = np.array([alpha_MRI_hydro(t) for t in T])
        cs2 = np.array([kB*T[i]/(mu*mH) for i in range(n)])
        alphas_stag, cs2_stag = update_stag_grid(alphas, cs2)

        St_dust = Q/(alphas_stag * cs2_stag)
        St_pebble = Q*10/(alphas*cs2)
        nu = alphas*cs2/Omega
        v_gas = calc_gas_vel(nu, sigma_gas, dist, dist_stag)
        v_dust = calc_dust_vel(St_dust, cs2_stag, v_gas, P)
        v_phi_gas = calc_v_phi_gas(P, cs2)
        v_peb = calc_pebble_vel(v_dust)
        D = 12 * nu / (X ** 2)
        f = (1.5 * X * sigma_gas)
        return (sigma_dust, sigma_gas, sigma_evap, sigma_pebble, sigma_planet, v_dust, v_gas, v_pebble, v_phi_gas, cs2, nu, D, f, alphas, alphas_stag, cs2_stag, St_dust, St_pebble)
    else:
        assert(t[0] == 0)
        sigma_gas = np.empty(n)
        sigma_dust = np.empty((3,n))
        sigma_pebble = np.zeros((3,n))
        sigma_evap_init = np.zeros((3,n))
        alphas = make_alpha_grid(cont)
        for i in range(n):
            sigma_gas[i] = sigma_in * AU / dist[i]
            if (sigma_gas[i]>sigma_max):
                sigma_gas[i] = sigma_max
            if (dist[i]/AU > 15):
                sigma_gas[i] = sigma_min
            sigma_dust[0,i] = sigma_gas[i] * 0.005 * cor_ratio
            sigma_dust[1,i] = sigma_gas[i] * 0.005 * calc_ratio
            sigma_dust[2,i] = sigma_gas[i] * 0.005 * for_ratio
        cs2, T, P, sigma_dust, sigma_evap, sigma_pebble = calc_thermal_struc(sigma_gas, sigma_dust, sigma_pebble, sigma_evap_init, alphas, Omega)
        calc_dist_omega_stag()
        alphas = np.array([alpha_MRI_hydro(t) for t in T])
        alphas_stag, cs2_stag = update_stag_grid(alphas, cs2)
        St_dust = Q/(alphas_stag * cs2_stag)
        St_pebble = Q*10/(alphas*cs2)
        nu = alphas*cs2/Omega
        v_gas = calc_gas_vel(nu, sigma_gas, dist, dist_stag)
        v_dust = calc_dust_vel(St_dust, cs2_stag, v_gas, P)
        v_phi_gas = calc_v_phi_gas(P, cs2)
        v_pebble = np.zeros(n)
        sigma_planet = np.zeros((3,n))
        D = 12 * nu / (X ** 2)
        f = (1.5 * X * sigma_gas)
    return (sigma_dust, sigma_gas, sigma_evap, sigma_pebble, sigma_planet, v_dust, v_gas, v_pebble, v_phi_gas, cs2, nu, D, f, alphas, alphas_stag, cs2_stag, St_dust, St_pebble)

In [19]:
# @numba.njit
def calc_pebble_vel(v_dust):
#     Stoke = 0.05
#     press_stag = [0.5*(press[i]+press[i-1]) for i in range(1,n)]
#     dPdr_stag = [(press[i] - press[i-1])/dist_stag[i-1] for i in range(1,n)]
#     rho_stag = press_stag/cs2_stag
#     v_dust = (Stoke/(1+(Stoke)**2)) * (1 / (rho_stag * Omega_stag))*dPdr_stag
#     v_dust = np.append(v_dust, 0)
#     v_dust = v_dust + v_gas
    return v_dust * 10

In [20]:
@numba.njit
def calc_pebble_growth(sigma_pebble, sigma_dust):
    tau = 1.988e-7 * 1e3*yrs2sec / Omega
    sigma_pebble += sigma_dust * dt / tau
    sigma_dust -= sigma_dust * dt / tau
    return (sigma_pebble, sigma_dust)

In [21]:
@numba.njit
def streaming_instability(sigma_planet, sigma_pebble, sigma_gas):
    total_pebble = sigma_pebble.sum(axis=0)
    pb_ratio = total_pebble/sigma_gas
    total_dust = sigma_dust.sum(axis=0)
    dg_ratio = total_dust/sigma_gas
    for i in range(n):
        if(pb_ratio[i] > 0.01 or dg_ratio[i] > 0.01):
            sigma_planet[:,i] += sigma_pebble[:,i]
            sigma_pebble[:,i] = np.zeros(3)
    return (sigma_planet, sigma_pebble)

In [22]:
# @numba.njit
def calc_planetesimal_growth(sigma_pebble, sigma_planet, sigma_dust, sigma_gas, St_pebble, v_phi_gas):
    sigma_planet, sigma_pebble = streaming_instability(sigma_planet, sigma_pebble, sigma_gas)
    if(sigma_planet.any() < 0):
        raise ValueError("Planets can't be negative.")
    if(sigma_pebble.any() < 0):
        raise ValueError("Can't have negative amount of pebbles.")
    total_planet = sigma_planet.sum(axis=0)
    if(total_planet.any() != 0):
        rho_p = sigma_pebble*Omega/(np.sqrt(cs2) * np.sqrt(2.0*np.pi))
        tau = 1.988e-7 * 1e3*yrs2sec / Omega
        v = v_phi_gas - v_phi_dust
        rad = 100e3
        mat_dens = 3000
        mass = 4/3 * np.pi * rad**3 * mat_dens
        total_mass = 2 * np.pi * dist * dr * total_planet
        num_planets = total_mass/mass
    #         print(total_mass)
        Rb = G*mass/np.square(v)
        tb = Rb/v
        tau_f = St_pebble/Omega
        r_acc = np.sqrt(np.sqrt(4*tau_f/tb) * Rb)
        accretion_rate = np.pi * np.square(r_acc) * rho_p * (v + Omega*r_acc)
        for i in range(3):
            for j in range(n):
                annulus = (2 * np.pi * dist[j] * dr)
                if(sigma_pebble[i,j] <= accretion_rate[i,j] * num_planets[j] / annulus):
                    sigma_planet[i,j] += sigma_pebble[i,j]
                    sigma_pebble[i,j] = 0
                else:
                    sigma_planet[i,j] += accretion_rate[i,j] * num_planets[j] / annulus
                    sigma_pebble[i,j] -= accretion_rate[i,j] * num_planets[j] / annulus
    return (sigma_planet, sigma_pebble)

In [23]:
@numba.njit
def calc_gas_evol(f, dt, D):
    """Outputs the surface density at a specific dt."""
    df_dt = np.empty(n)
    for j in range(1, n-1):
        dX1 = X[j] - X[j-1]
        dX2 = X[j+1] - X[j]
        D1 = 0.5 * (D[j] + D[j-1])
        D2 = 0.5 * (D[j+1] + D[j])
        df_dt[j] = D1 * ((-(f[j] - f[j-1])/dX1**2)) + D2 * ((f[j+1]-f[j])/dX2**2)
    dX_final = X[-1]-X[-2]
    dX_in = X[1]-X[0]
    df_dt[0] = D[0] * (-(f[0] - f_in)/dX_in**2 + (f[1]-f[0])/dX_in**2)
    df_dt[n-1] = D[n-1] * (-(f[n-1] - f[n-2])/dX_final**2 + (f_out-f[n-1])/dX_final**2)
    f_new = f + df_dt * dt
    sigma_at_time = np.array([2*f_new[k]/(3*X[k]) for k in range(n)])
    return (f_new, sigma_at_time)

In [24]:
# @numba.njit
def calc_disk_mass(sigma):
    """Calculate the mass at a specific dt"""
    disk_mass = 0
    for j in range(n):
        disk_mass += 2 * np.pi * dist[j] * dr * sigma_gas[j]
        disk_mass += 2 * np.pi * dist[j] * dr * sigma_dust[j]
#     print(f'The mass of the disk is {disk_mass}.')
    return disk_mass

In [25]:
def save2dir():
    output_dir = 'output/'
    filename = 'output/disk_'
    return (output_dir, filename)

In [26]:
#Time Evolution Main
folder='output'
sigma_dust, sigma_gas, sigma_evap, sigma_pebble, sigma_planet, v_dust, v_gas, v_pebble, v_phi_gas, cs2, nu, D, f, alphas, alphas_stag, cs2_stag, St_dust, St_pebble = calc_init_params()
output_dir, filename = save2dir()
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for i in range(len(t)):
    f_new, sigma_gas = calc_gas_evol(f, dt, D)
    sigma_pebble, sigma_dust = calc_pebble_growth(sigma_pebble, sigma_dust)
    sigma_planet, sigma_pebble = calc_planetesimal_growth(sigma_pebble, sigma_planet, sigma_dust, sigma_gas, St_pebble, v_phi_gas)
    for el in range(3):
        sigma_dust[el,:] = calc_evol(sigma_gas, sigma_dust[el,:], nu, v_dust, dist, dt)
        sigma_pebble[el,:] = calc_evol(sigma_gas, sigma_pebble[el,:], nu, v_pebble, dist, dt)
        sigma_evap[el,:] = calc_evol(sigma_gas, sigma_evap[el,:], nu, v_gas, dist, dt)
    cs2, T, P, sigma_dust, sigma_evap, sigma_pebble = calc_thermal_struc(sigma_gas, sigma_dust, sigma_pebble, sigma_evap, alphas, Omega)
    alphas = np.array([alpha_MRI_hydro(t) for t in T])
    alphas_stag, cs2_stag = update_stag_grid(alphas, cs2)
    nu = update_nu(cs2, alphas)
    D = update_D(nu)
    St_dust, St_pebble = update_St(cs2, cs2_stag, alphas, alphas_stag)
    v_dust = calc_dust_vel(St_dust, cs2_stag, v_gas, P)
    v_pebble = calc_pebble_vel(v_dust)
    v_gas = calc_gas_vel(nu, sigma_gas, dist, dist_stag)
    v_phi_gas = calc_v_phi_gas(P, cs2)
    f = f_new
    if (i%(t_save_interval/dyr) == 0):
        output = np.transpose(np.vstack([dist/AU, sigma_gas, sigma_dust, sigma_pebble, sigma_planet, T, P]))
        np.savetxt(filename + str(int(i*dyr)) + '.txt', output, delimiter=',', newline='\n')# save results to file

In [108]:
def plot(filename):
    %matplotlib
    t_plot = [7e4, 9e4]
    markers = ['-',':']
    colors = ['fuchsia', 'navy']
    assert(len(markers) == len(t_plot))
    for idx,time in enumerate(t_plot):
        df = pd.read_csv(filename + '/disk_' + str(int(time)) + '.txt', delimiter=',', header=None)
        sigma_gas_plot = df[1].to_numpy().astype(np.float)
        cor_dust = df[2].to_numpy().astype(np.float)
        calc_dust = df[3].to_numpy().astype(np.float)
        for_dust = df[4].to_numpy().astype(np.float)
        cor_evap = df[5].to_numpy().astype(np.float)
        calc_evap = df[6].to_numpy().astype(np.float)
        for_evap = df[7].to_numpy().astype(np.float)
        cor_peb = df[8].to_numpy().astype(np.float)
        calc_peb = df[9].to_numpy().astype(np.float)
        for_peb = df[10].to_numpy().astype(np.float)
        cor_planet = df[11].to_numpy().astype(np.float)
        calc_planet = df[12].to_numpy().astype(np.float)
        for_planet = df[13].to_numpy().astype(np.float)
        v_dust = df[14].to_numpy().astype(np.float)
        temp_plot = df[15].to_numpy().astype(np.float)
        press_plot = df[16].to_numpy().astype(np.float)
        
        alphas = np.array([alpha_MRI_hydro(t) for t in T])
        cs2 = np.array([kB*temp_plot[i]/(mu*mH) for i in range(n)])
        alphas_stag, cs2_stag = update_stag_grid(alphas, cs2)
        
        sigma_dust_plot = cor_dust+calc_dust+for_dust
        sigma_peb_plot = cor_peb+calc_peb+for_peb
        sigma_planet_plot = cor_planet+calc_planet+for_planet
        planet_mass = sigma_planet_plot * (2 * np.pi* dist * dr)
        pg_ratio = sigma_peb_plot/sigma_gas_plot
        dg_ratio = sigma_dust_plot/sigma_gas_plot
        pdg_ratio = (sigma_peb_plot + sigma_dust_plot)/sigma_gas_plot
        total_al = cor_dust+cor_evap+cor_peb
        total_calc = calc_dust+calc_evap+calc_peb
        total_mg = for_dust+for_evap+for_peb
        al_mg_ratio =  total_al/(total_mg * (calc_ratio/for_ratio))
        calc_mg_ratio = total_calc/(total_mg * (calc_ratio/for_ratio))
        pebbles = [cor_peb, calc_peb, for_peb]
#         print(sigma_planet_plot)
        eps = 1e-20
        for lst in pebbles:
            low_vals = lst < eps  # Where values are low
            lst[low_vals] = 0
        if(idx==0):
            fig, axes = plt.subplots(3,3, figsize=(12,7.5))
        ax1 = axes[0][0]
        ax2 = axes[1][0]
        ax3 = axes[0][1]
        ax4 = axes[1][1]
        ax5 = axes[0][2]
        ax6 = axes[1][2]
        ax7 = axes[2][0]
        ax8 = axes[2][1]
        ax9 = axes[2][2]
        for row in axes:
            for grid in row:
                grid.set_xlim([0,5])
                grid.tick_params(direction='in', length=3,   which='major', top=True, right=True)
                grid.tick_params(direction='in', length=1.5, which='minor', top=True, right=True)
                if(grid == ax7 or grid == ax8 or grid == ax9):
                    grid.set_xlabel('Distance (AU)')
                if(grid == ax1 or grid==ax2 or grid==ax4 or grid==ax7):
                    grid.set_yscale('log')
                    grid.set_ylabel('Surface Density [kg/m$^3$]')
        fig.suptitle(f'Total Evolution at time = {int(t_plot[-2])} vs {int(t_plot[-1])} years', fontsize=14)
        ax1.set_title('Gas')
        ax1.plot(dist/AU, sigma_gas_plot,color='lightseagreen',linestyle = markers[idx],label=int(time))
        ax1.legend(fontsize='small', loc='lower left') 
        ax2.set_title('Dust')
        ax2.plot(dist/AU, cor_dust, label='Aluminum', color='green', linestyle = markers[idx])
        ax2.plot(dist/AU, calc_dust, label='Calcium', color='red',linestyle = markers[idx])
        ax2.plot(dist/AU, for_dust, label='Magnesium', color='blue',linestyle = markers[idx])
        ax3.set_title(f'Pebble and Dust to Gas Ratio')
        ax3.set_ylabel('Solid/Gas')
        ax3.set_yscale('log')
        ax3.set_ylim(1e-4,1e0)
        ax3.plot(dist/AU, dg_ratio, label='dust/gas', color='blue',linestyle = markers[idx])
        ax3.plot(dist/AU, pg_ratio, label='pebble/gas', color='green',linestyle = markers[idx])
        ax3.plot(dist/AU, pdg_ratio,label='(dust+pebble)/gas', color='red',linestyle = markers[idx])
        ax4.set_title(f'Pebbles')
#         ax4.set_ylim(1e-4,1e2)
        ax4.plot(dist/AU, cor_peb, label='Aluminum', color='green',linestyle = markers[idx])
        ax4.plot(dist/AU, calc_peb, label='Calcium', color='red',linestyle = markers[idx])
        ax4.plot(dist/AU, for_peb, label='Magnesium', color='blue',linestyle = markers[idx])
        ax5.set_title(f'Temperature Evolution')
        ax5.set_ylabel('Temperature [K]')
        ax5.set_yscale('log')
        ax5.set_ylim(1e0,1e4)
        ax5.plot(dist/AU, temp_plot, color='olive', linestyle=markers[idx], label=int(time))
        ax5.legend(fontsize='small', loc='lower left') 
        ax6.set_title(f'Pressure Evolution')
        ax6.set_ylabel('Pressure [Pa]')
        ax6.set_yscale('log')
        ax6.set_ylim(1e-2,1e4)
        ax6.plot(dist/AU, press_plot, color='purple', linestyle = markers[idx], label=int(time))
        ax6.legend(fontsize='small', loc='lower left') 
        ax7.set_title(f'Planetesimal')
        ax7.set_yscale('log')
        ax7.plot(dist/AU, cor_planet, label='Aluminum', color='green',linestyle = markers[idx])
        ax7.plot(dist/AU, calc_planet, label='Calcium', color='red',linestyle = markers[idx])
        ax7.plot(dist/AU, for_planet, label='Magnesium', color='blue',linestyle = markers[idx])
        ax8.set_title(f'Planetesimal Mass')
        ax8.set_ylabel('Mass [kg]')
        ax8.set_yscale('log')
        ax8.plot(dist/AU, planet_mass, color=colors[idx], linestyle = markers[idx], label=int(time))
        ax8.legend(fontsize='small', loc='upper right') 
        ax9.set_title(f'Elemental Ratios')
        ax9.set_ylabel('(Al or Ca) / Mg')
        ax9.set_yscale('log')
        ax9.set(ylim=(1e-2,5e1))
        ax9.plot(dist/AU, al_mg_ratio, color='midnightblue', linestyle = markers[idx], label='Al/Mg')
        ax9.plot(dist/AU, calc_mg_ratio, color='deeppink', linestyle = markers[idx], label='Ca/Mg')
        for row in axes:
            for grid in row:
                if(idx == 0):
                    if(grid==ax2 or grid==ax4 or grid==ax7 or grid == ax9):
                        grid.legend(fontsize='small', loc='lower right')
                    elif(grid==ax3):
                        grid.legend(fontsize='small', loc='upper right')
                    
    plt.show()
    plt.tight_layout()
    plt.savefig(f'./{filename}_{int(t_plot[-2])}_{int(t_plot[-1])}.pdf', transparent=True)

In [109]:
plot('tau1e4_St5e-2_alphaMRI_SI1e-2_PA')

Using matplotlib backend: MacOSX


In [124]:
def plot_planet_growth(files):
    %matplotlib
    t_plot = np.arange(61e3, 90e3, 1e3)
    colors = ['fuchsia', 'navy']
    planet_masses = [[] for i in range(len(files))]
    stokes = ['Stx10','St5e-2']
    for idx, filename in enumerate(files):
        for time in t_plot:
            df = pd.read_csv(filename + '/disk_' + str(int(time)) + '.txt', delimiter=',', header=None)
            cor_planet = df[11].to_numpy().astype(np.float)
            calc_planet = df[12].to_numpy().astype(np.float)
            for_planet = df[13].to_numpy().astype(np.float)
            sigma_planet_plot = cor_planet+calc_planet+for_planet
            mass = sum(sigma_planet_plot * (2 * np.pi* dist * dr))
            planet_masses[idx].append(mass)
        if(idx==0):
            fig = plt.figure(figsize=(12,7.5))
        plt.tick_params(direction='in', length=3,   which='major', top=True, right=True)
        plt.tick_params(direction='in', length=1.5, which='minor', top=True, right=True)
        fig.suptitle(f'Planetesimal Growth Rate', fontsize=14)
        plt.ylabel('Mass [kg]')
        plt.xlabel('Time [Kyr]')
        plt.yscale('log')
    #     plt.ylim(5e22, 1e24)
        plt.plot(t_plot/1000, planet_masses[idx], color=colors[idx], label=stokes[idx])
    plt.legend()
    plt.show()
    plt.tight_layout()
    plt.savefig(f'./comp_stokes_planet_growth_rate_{filename}.pdf', transparent=True)

In [125]:
plot_planet_growth(['tau1e4_Stx10_alphaMRI_SI1e-2_PA','tau1e4_St5e-2_alphaMRI_SI1e-2_PA'])

Using matplotlib backend: MacOSX


In [32]:
def compare(files):
    %matplotlib
    t_plot = [1e4,2e4,3e4,4e4,5e4,6e4,7e4,8e4,9e4,1e5]
    markers = ['-',':']
    comp_vals = ['St 0.05','St x 10']
    assert(len(files) == len(markers))
    for time in t_plot:
        for idx, file in enumerate(files):
            df = pd.read_csv(file + '/disk_' + str(int(time)) + '.txt', delimiter=',', header=None)
            sigma_gas_plot = df[1].to_numpy().astype(np.float)
            cor_dust = df[2].to_numpy().astype(np.float)
            calc_dust = df[3].to_numpy().astype(np.float)
            for_dust = df[4].to_numpy().astype(np.float)
            cor_evap = df[5].to_numpy().astype(np.float)
            calc_evap = df[6].to_numpy().astype(np.float)
            for_evap = df[7].to_numpy().astype(np.float)
            cor_peb = df[8].to_numpy().astype(np.float)
            calc_peb = df[9].to_numpy().astype(np.float)
            for_peb = df[10].to_numpy().astype(np.float)
            temp_plot = df[12].to_numpy().astype(np.float)
            press_plot = df[13].to_numpy().astype(np.float)
            sigma_evap_plot = cor_evap+calc_evap+for_evap
            sigma_dust_plot = cor_dust+calc_dust+for_dust
            sigma_peb_plot = cor_peb+calc_peb+for_peb
            total_al = cor_dust+cor_evap+cor_peb
            total_calc = calc_dust+calc_evap+calc_peb
            total_mg = for_dust+for_evap+for_peb
            al_mg_ratio =  total_al/(total_mg * (calc_ratio/for_ratio))
            calc_mg_ratio = total_calc/(total_mg * (calc_ratio/for_ratio))
            pg_ratio = sigma_peb_plot/sigma_gas_plot
            dg_ratio = sigma_dust_plot/sigma_gas_plot
            pdg_ratio = (sigma_peb_plot + sigma_dust_plot)/sigma_gas_plot
            if(idx == 0):
                fig, axes = plt.subplots(3,3, figsize=(12,7.5))
            ax1 = axes[0][0]
            ax2 = axes[1][0]
            ax3 = axes[0][1]
            ax4 = axes[1][1]
            ax5 = axes[1][2]
            ax6 = axes[0][2]
            ax7 = axes[2][1]
            for row in axes:
                for grid in row:
                    grid.set_xlim([0,5])
                    grid.tick_params(direction='in', length=3,   which='major', top=True, right=True)
                    grid.tick_params(direction='in', length=1.5, which='minor', top=True, right=True)
                    if(grid == ax2 or grid == ax4 or grid == ax5):
                        grid.set_xlabel('Distance (AU)')
                    if(ax1 or ax2 or ax4):
                        grid.set_yscale('log')
                        grid.set_ylabel('Surface Density [kg/m$^3$]')
            fig.suptitle(f'Total Evolution at time = {int(time)} years', fontsize=14)
            ax1.set_title('Gas')
            ax1.plot(dist/AU, sigma_gas_plot,color='lightseagreen',linestyle = markers[idx],label=comp_vals[idx])
            ax1.legend(fontsize='small', loc='lower left') 
            ax2.set_title('Dust')
            ax2.plot(dist/AU, cor_dust, label='Aluminum', color='green', linestyle = markers[idx])
            ax2.plot(dist/AU, calc_dust, label='Calcium', color='red',linestyle = markers[idx])
            ax2.plot(dist/AU, for_dust, label='Magnesium', color='blue',linestyle = markers[idx])
            ax3.set_title(f'Pebble and Dust to Gas Ratio')
            ax3.set_ylabel('Solid/Gas')
            ax3.set_yscale('log')
            ax3.set_ylim(1e-4,1e0)
            ax3.plot(dist/AU, dg_ratio, label='dust/gas', color='blue',linestyle = markers[idx])
            ax3.plot(dist/AU, pg_ratio, label='pebble/gas', color='green',linestyle = markers[idx])
            ax3.plot(dist/AU, pdg_ratio,label='(dust+pebble)/gas', color='red',linestyle = markers[idx])
            ax4.set_title(f'Pebbles')
            ax4.plot(dist/AU, cor_peb, label='Aluminum', color='green',linestyle = markers[idx])
            ax4.plot(dist/AU, calc_peb, label='Calcium', color='red',linestyle = markers[idx])
            ax4.plot(dist/AU, for_peb, label='Magnesium', color='blue',linestyle = markers[idx])
            ax5.set_title(f'Temperature Evolution')
            ax5.set_ylabel('Temperature [K]')
            ax5.set_yscale('log')
#             ax5.set_ylim(1e0,1e5)
            ax5.plot(dist/AU, temp_plot, color='olive', linestyle=markers[idx], label=comp_vals[idx])
            ax5.legend(fontsize='small', loc='lower left') 
            ax6.set_title(f'Pressure Evolution')
            ax6.set_ylabel('Pressure [Pa]')
            ax6.set_yscale('log')
            ax6.set_ylim(1e-2,1e4)
            ax6.plot(dist/AU, press_plot, color='purple', linestyle = markers[idx], label=comp_vals[idx])
            ax6.legend(fontsize='small', loc='lower left') 
            ax7.set_title(f'Elemental Ratios')
            ax7.set_ylabel('(Al or Ca) / Mg')
            ax7.set_yscale('log')
            ax7.set(ylim=(1e-2,5e1))
            ax7.plot(dist/AU, al_mg_ratio, color='midnightblue', linestyle = markers[idx], label='Al/Mg')
            ax7.plot(dist/AU, calc_mg_ratio, color='deeppink', linestyle = markers[idx], label='Ca/Mg')
            for row in axes:
                for grid in row:
                    if((ax2 or ax3 or ax4 or ax7) and (idx == 0)):
                        grid.legend(fontsize='small', loc='lower right')
#             plt.show()
            plt.tight_layout()
            plt.savefig(f'./St_5e-2vsx10_tau1e5{int(time)}.pdf', transparent=True)

In [33]:
# files = ['tau1e5_St5e-2_alphaMRI','tau1e5_Stx10_alphaMRI']
# compare(files)

In [34]:

#         cm = pylab.get_cmap('jet') #gist_rainbow
#         plt.rcParams['font.family']='sans-serif'
#         plt.rcParams['axes.linewidth']= 1.0
#         plt.rcParams['font.size']     = 14
#         plt.rcParams['figure.figsize']= 4*1.414, 4
#         rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
#         rc('text',usetex=False)
#         rc('text.latex', preamble=r'\usepackage{sfmath}')